# Basic usage of Specam

This notebook gives an overview of how to use this package including
- Defining spectral sensor/camera properties
- Generating mock data (with noise) using simple functions or experimental data for emissivity 
- Estimating temperature from this data using multiple methods and comparing to known value

In [ ]:
%load_ext autoreload
%autoreload 2

from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.interpolate import interp1d

import arviz as az

import specam
from specam.constants import abs0

import pytensor
pytensor.config.cxx = '/usr/bin/clang++'


RANDOM_SEED = 8927
rng = np.random.default_rng(RANDOM_SEED)
az.style.library["arviz-darkgrid"]['figure.constrained_layout.use'] = False
az.style.use("arviz-darkgrid")

%matplotlib widget
# %matplotlib inline

## Camera definition
A camera is defined by the frequencies it measures data at, this can be set either by evenly spaced values in a range or by a list of values. A signal to noise ratio can be set to calculate noise relative to maximum signal when generating spectral data. Further key-value pairs can also be specified and are stored in the `props` dictionary within a camera to be used later in analysis. Here a set of example cameras are created based on the specifications of available cameras.

In [ ]:
camera_specs = {
    # 'telops_demo': {
    #     'kind': 'vals',
    #     'vals': np.array([3.69e-6, 3.80e-6, 3.96e-6, 4.09e-6]),
    # },

    'far_fmpi': {   # Far FMPI — 300 – 2000°C
        'kind': 'linspace',
        'start': 900e-9,
        'stop': 1700e-9,
        'num': 250,
        'signal_noise_ratio': 1000,
        'colour': 'blue',
    },
    'far_fmp2': {   # Far FMP2 — 800 – 2500°C
        'kind': 'linspace',
        'start': 500e-9,
        'stop': 1000e-9,
        'num': 500,
        'signal_noise_ratio': 1000,
        'colour': 'blue',
    },

    'specim_fx10': {
        'kind': 'linspace',
        'start': 400e-9,
        'stop': 1000e-9,
        'num': 224,
        'signal_noise_ratio': 1000,
        'colour': 'blue',
        'label': '',
    },
    'specim_fx17': {
        'kind': 'linspace',
        'start': 900e-9,
        'stop': 1700e-9,
        'num': 224,
        'signal_noise_ratio': 1000,
        'colour': 'red',
    },
    'specim_fx50': {
        'kind': 'linspace',
        'start': 2700e-9,
        'stop': 5300e-9,
        'num': 154,
        'signal_noise_ratio': 1000,
        'colour': 'green',
    },
}

cameras = {
    name: specam.Camera.create(name, **specs) 
    for name, specs in camera_specs.items()
}


## Emissivity curve from experiment
The material emissivity used to generate representative spectral data can be taken from an experimentally measured curve. Several emissivity curves for tungsten have been taken from literature [1] and are plotted below, there is a range of values and curve shape which could be due to differences in surface finish, measurement temperature and measurement technique. The formatted data is available at https://git.ccfe.ac.uk/mlobb/hyper-spectralproject . Black vertical lines have been plotted at the minimum and maximum wavelength of all the cameras considered to aid choosing a suitable curve.

[1] Touloukian, Y S, and D P DeWitt. ‘Thermophysical Properties of Matter - the TPRC Data Series. Volume 7. Thermal Radiative Properties - Metallic Elements and Alloys. (Reannouncement). Data Book’. United States, 1 January 1970. https://www.osti.gov/biblio/5305606.

In [ ]:
excel_path = Path('/Users/xg3401/software/hyper-spectralproject/Data/'
                  'Processed/Original/Tungsten P864.xlsx')
excel_data = pd.read_excel(excel_path, None)

emissivity_curves = {}
for curve_name, data in excel_data.items():
    curve_i = int(curve_name.split()[1])

    temp = data['Temperature K'][0]
    assert np.all(data['Temperature K'] == temp)

    key = (curve_i, int(temp))
    emissivity_curves[key] = data[['Wavelength', 'Emissivity']].to_numpy()

In [ ]:
highlight_i = 0
wavelength_range = np.array([4e-7, 5e-6])

fig, ax = plt.subplots(1, 1)
ax.vlines(wavelength_range * 1e6, 0, 1, color='k', lw=2)

for i, (label, curve) in enumerate(emissivity_curves.items()):
    if i == highlight_i:
        ax.plot(curve[:, 0] * 1e6, curve[:, 1], '-*', zorder=10, lw=2, color='k')
        print('Highlighted `curve_key`:', label)
    else:
        ax.plot(curve[:, 0] * 1e6, curve[:, 1], lw=0.5)

ax.set_xlabel("Wavelength ($\mu$m)")
ax.set_ylabel("Emissivity")
ax.set_ylim([0, 1])

The selected emissivity is now plotted with 3rd order polynomial function fitted, showing if this function can be used to correctly interpolate the (limited) data in the required wavelength range.

In [ ]:
curve_key = (1, 1605)
wavelength_range = np.array([4e-7, 5e-6])

curve = emissivity_curves[curve_key]

wavelength_vals = np.linspace(*wavelength_range, 100)
curve_interp = interp1d(curve[:, 0], curve[:, 1], kind=3)

fig, axes = plt.subplots(2, 1, constrained_layout=True, figsize=(4, 6))

ax = axes[0]
ax.vlines(wavelength_range * 1e6, 0, 1, color='k', lw=2)
ax.plot(curve[:, 0] * 1e6, curve[:, 1], '*', color='k')
ax.plot(wavelength_vals * 1e6, curve_interp(wavelength_vals), '-', color='b')
ax.set_xlabel("Wavelength ($\mu$m)")
ax.set_ylabel(" Emissivity")
ax.set_ylim([0, 1])

ax = axes[1]
ax.vlines(wavelength_range, 0, 1, color='k', lw=2)
ax.plot(curve[:, 0], curve[:, 1], '*', color='k')
ax.plot(wavelength_vals, curve_interp(wavelength_vals), '-', color='b')
ax.set_xscale('log')
ax.set_xlabel("Wavelength")
ax.set_ylabel(" Emissivity")
ax.set_ylim([0, 1])


Once a reasonable emissivity curve and interpolation method has been selected, this can be wrapped to create function for data generation.

In [ ]:
# Create intensity function based on data emissivity

excel_path = Path('/Users/xg3401/software/hyper-spectralproject/Data/'
                  'Processed/Original/Tungsten P864.xlsx')
curve_key = (1, 1605)
data_emissivity = specam.utils.load_data_emissivity(
    excel_path, curve_key, kind=3, fill_value="extrapolate"
)

def data_intensity_func(lam, T, lam_0=None, lam_inf=None):
    return data_emissivity(lam) * specam.models.planck_eqn(lam, T)

def data_intensity_func_log(lam, T, lam_0=None, lam_inf=None):
    return np.log(data_emissivity(lam)) + specam.models.planck_eqn_log(lam, T)


## Test data generation
Test spectral data is generated to assess temperature determination methods. `T_N` spectrums will be generated in the temperature range (`T_0`, `T_inf`) with Gaussian noise added with standard deviation `noise_sigma` or can be determined from the signal to noise ratio defined with the camera. Emissivity function is defined either the data function from above or a linear function of wavelength, with parameters `C` and `D` defining the gradient and intercept respectively,

$ \epsilon(\lambda) = D - C\frac{\lambda - \lambda_0}{\lambda_\infty - \lambda_0}$,

where ($\lambda_0$, $\lambda_\infty$) is the wavelength range and $C$ and $D$ must be in range (0, 1) to ensure emissivity varies in the same range.

In [ ]:
camera_name = 'specim_fx17'
camera = cameras[camera_name]

T_0, T_inf = 200+abs0, 3000+abs0
T_N = 10

noise_sigma = None  # signal to noise ratio will be used
# noise_sigma = 0.1
# noise_sigma = 1e6

# Data emissivity function
# intensity_func_params = (data_intensity_func, data_intensity_func_log, {})

# Linear emissivity function
C, D = 0.05, 0.9
# C, D = 0., 0.8    # constant emissivity
intensity_func_params = (
    specam.models.intensity_func, 
    specam.models.intensity_func_log, 
    {'C': C, 'D': D}
)

camera.create_test_data(T_0, T_inf, T_N, *intensity_func_params, noise_sigma)


## Fit data for temperature

In [ ]:
camera.fit_test_data('lmfit')
camera.fit_test_data('scipy')
# camera.fit_test_data('pymc')
camera.fit_test_data('ratio', polyorder=0, combinations=5)

In [ ]:
# camera = cameras['specim_swir']
camera = camera
i_T = None
model_names = [
    'lmfit', 
    'scipy',
    'ratio',
    # 'pymc',
]

plot = camera.plot_spectrum(
    i_batch=i_T, 
    # log=0, 
    result_names=model_names, 
    # plot_diff=True, 
    # manual_fit=True,
)
# camera.plot_spectrum(i_T, log=False, plot_result_names=['lmfit', 'pymc'])
# camera.plot_spectrum(i_T, log=True, plot_result_names=['lmfit', 'pymc'])

plot.add_legend()

In [ ]:
camera = camera
model_names = [
    'lmfit', 
    'scipy',
    'ratio',
    # 'pymc',
]


fig, axes = plt.subplots(2, 2, figsize=(7, 6), constrained_layout=True)
axes = axes.flat
fig.suptitle(camera.name)

T_true = camera.test_data['temperature'] - abs0

ax = axes[0]
ax.plot(T_true, label='True', color='k')
for model_name in model_names:
    ax.plot(camera.results[model_name]['T'] - abs0, label=model_name)

ax.set_xlabel('Position')
ax.set_ylabel('Temperature ($^\circ$C)')
ax.legend()


ax = axes[1]
# ax.plot(
#     T_true, 
#     camera.test_data['noise_sigma'], 
#     label='True', color='k'
# )
for model_name in model_names:
    model_data = camera.results[model_name]
    variance = model_data.get('covar')
    if variance is None:
        variance = model_data.get('sigma')
    else:
        variance = variance[:, 0, 0]
    if variance is None:
        variance = model_data.get('T_std')**2
    if variance is None:
        continue
    ax.plot(T_true, variance, label=model_name)

ax.set_xlabel('Temperature ($^\circ$C)')
ax.set_ylabel('Estimated variance')
ax.legend()


ax = axes[2]
C_true = camera.test_data.intensity_params.get('C')
if C_true is not None:
    ax.hlines(C_true, T_true[0], T_true[-1], label='True', color='k')
for model_name in model_names:
    C_fit = camera.results[model_name].get('C')
    if C_fit is not None:
        ax.plot(T_true, C_fit, label=model_name)

ax.set_xlabel('Temperature ($^\circ$C)')
ax.set_ylabel('C')
ax.legend()


ax = axes[3]
D_true = camera.test_data.intensity_params.get('D')
if D_true is not None:
    ax.hlines(D_true, T_true[0], T_true[-1], label='True', color='k')
for model_name in model_names:
    D_fit = camera.results[model_name].get('D')
    if D_fit is not None:
        ax.plot(T_true, D_fit, label=model_name)

ax.set_xlabel('Temperature ($^\circ$C)')
ax.set_ylabel('D')
ax.legend()


In [ ]:
fig, axes = plt.subplots(1, 1, figsize=(6, 3), constrained_layout=True)
fig.suptitle(camera.name)

T_true = camera.test_data['temperature']

ax = axes
for model_name in model_names:
    err = camera.results[model_name]['T'] - T_true
    # err /= T_true / 100   # relative error
    ax.plot(T_true - abs0, err, '*-', label=model_name)

ax.set_xlabel('Temperature ($^\circ$C)')
ax.set_ylabel('Temperature error (K)')
ax.set_ylim([-50, 50])
ax.legend()

# C_test = camera.test_data['intensity_params'].get('C')
# if C_test is not None:
#     ax = axes[1]
#     for model_name in model_names:
#         ax.plot(camera.test_data['T']-abs0, 
#                 camera.results[model_name]['C'] - C_test, '*-', 
#                 label=model_name)

#     ax.set_xlabel('Temperature ($^\circ$C)')
#     ax.set_ylabel('C error')
#     ax.set_ylim([-0.01, 0.01])
#     ax.legend()

# D_test = camera.test_data['intensity_params'].get('D')
# if D_test is not None:
#     ax = axes[2]
#     for model_name in model_names:
#         ax.plot(camera.test_data['T']-abs0, 
#                 camera.results[model_name]['D'] - D_test, '*-',
#                 label=model_name)
        
#     ax.set_xlabel('Temperature ($^\circ$C)')
#     ax.set_ylabel('D error')
#     ax.set_ylim([-0.01, 0.01])

#     ax.legend()


## Apply to multiple cameras

In [ ]:
T_0, T_inf = 200 + abs0, 3000 + abs0
T_N = 20
C, D = 0.05, 0.9
intensity_func_params = (
    specam.models.intensity_func, 
    specam.models.intensity_func_log, 
    {'C': C, 'D': D}
)

for camera in cameras.values():
    camera.create_test_data(T_0, T_inf, T_N, *intensity_func_params)
    camera.fit_test_data('lmfit')


In [ ]:
fig, axes = plt.subplots(
    len(cameras), 3, 
    figsize=(8, 2*len(cameras)), 
    constrained_layout=True,
)

for camera, ax_row in zip(cameras.values(), axes):
    T_true = camera.test_data['temperature'] - abs0
    C_true = camera.test_data.intensity_params['C']
    D_true = camera.test_data.intensity_params['D']

    ax = ax_row[0]
    ax.plot(T_true)
    ax.plot(camera.results['lmfit']['T'] - abs0)
    ax.set_ylabel('Temperature ($^\circ$C)')

    ax = ax_row[1]
    ax.set_title(camera.name)
    ax.hlines(C_true, T_true[0], T_true[-1])
    ax.plot(T_true, camera.results['lmfit']['C'])
    ax.set_ylabel('C')

    ax = ax_row[2]
    ax.hlines(D_true, T_true[0], T_true[-1])
    ax.plot(T_true, camera.results['lmfit']['D'])
    ax.set_ylabel('D')

ax_row[0].set_xlabel('Position')
ax_row[1].set_xlabel('Temperature ($^\circ$C)')
ax_row[2].set_xlabel('Temperature ($^\circ$C)')


In [ ]:
plt.figure()
ax = plt.gca()

for camera in cameras.values():
    ax.plot(
        camera.test_data['temperature'] - abs0, 
        camera.results['lmfit']['T'] - camera.test_data['temperature'], 
        label=camera.name
)

ax.set_xlabel('Temperature ($^\circ$C)')
ax.set_ylabel('Temperature error (K)')

ax.set_ylim([-50, 50])

plt.legend()